## 简答题

1. TensorFlow 是否可以简单替代 NumPy？两者之间的主要区别是什么？

不能，numpy主要用于数据预处理，科学计算、小规模数值计算；tensorflow用于机器学习模型训练，需要硬件加速计算

2. 使用 `tf.range(10)` 和 `tf.constant(np.arange(10))` 是否会得到相同的结果？

结果相同但是数据类型不同

3. 可以通过编写函数或继承 `tf.keras.losses.Loss` 来定义自定义损失函数。两种方法分别应该在什么时候使用？

普通函数：用于简单的数学运算，无状态和配置参数
Loss子类：保存状态、配置参数、复杂逻辑、序列化需求

4. 可以直接在函数中定义自定义指标或采用 `tf.keras.metrics.Metric` 子类。两种方法分别应该在什么时候使用？

普通函数：简单指标，无状态需求
继承子类：需要维护状态，复杂指标计算

5. 什么时候应该自定义层而不是自定义模型？

自定义层：创建可重用的基础组件
自定义模型：定义完整的模型架构、训练逻辑、自定义方法

6. 有哪些示例需要编写自定义训练循环？

不同任务的不同权重和损失、需要动态调整的，自定义优化器的

7. 自定义 Keras 组件中可以包含任意 Python 代码，还是必须转换为 TF 函数？

在eager模式下：可以正常运行所有Python代码
在graph模式下，必须使用TF函数转换

8. 如果要将函数转换为 TF 函数，应避免哪些主要模式？

使用python原生类型，python的打印语句，文件操作
修改外部状态
使用可变数据类型

9. 何时需要创建动态 Keras 模型？ 如何动态创建Keras模型？为什么不是所有模型都动态化？

动态调整层数、有分支语句的时候、有依赖的形状的时候

定义继承自tf.keras.Model模型的类在其中编写build\call函数

因为静态图更好优化，性能更好，方便调试和理解，更用于保存

## 编程题

1. 实现一个执行层归一化的自定义层：
    - a. `build()` 方法应定义两个可训练的权重 α 和 β，它们的形状均为 `input_shape[-1:]`，数据类型为 `tf.float32`。α 应该用 1 初始化，而 β 必须用 0 初始化。
    - b. `call()` 方法应计算每个实例特征的均值和标准差。为此，可以使用 `tf.nn.moments(inputs, axes=-1, keepdims=True)`，它返回同一实例的均值 μ 和方差 σ²（计算方差的平方根便可获得标准差）。然后，该函数应计算并返回
      $$
      \alpha \otimes \frac{(X-\mu)}{(\sigma+\epsilon)} + \beta
      $$
      其中 ε 是表示项精度的一个常量（避免被零除的小常数，例如 0.001）,$\otimes$表示逐个元素相乘
    - c. 确保自定义层产生与tf.keras.layers.LayerNormalization层相同（或几乎相同）的输出。



In [15]:
import tensorflow as tf
from sklearn.model_selection import train_test_split


class MyDense(tf.keras.layers.Layer):
    def __init__(self,epsilon= 1e-3,  **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        self.alpha = self.add_weight( name="alpha",shape=input_shape[-1:], initializer='ones', dtype=tf.float32)
        self.beta = self.add_weight( name="beta",shape=input_shape[-1:], initializer='zeros', dtype= tf.float32)
        super().build(input_shape)  # 这种写法更简洁

    def call(self, inputs, *args, **kwargs):
        # 计算每个实例特征的均值和标准差
        # tf.nn.moments(inputs, axes=-1,keepdims=True) 返回同一实例的均值和方差
        self.mu,self.sigma = tf.nn.moments(inputs, axes=-1,keepdims=True)

        return self.alpha * (inputs - self.mu)/(self.sigma+self.epsilon) + self.beta

    def get_config(self):
        base_config = super().get_config()
        return {**base_config,"epsilon":self.epsilon}

In [20]:
# 测试函数
def test_layer_normalization():
    print("测试层归一化实现...")

    # 创建测试数据
    batch_size, seq_length, hidden_size = 4, 6, 8
    test_input = tf.random.normal([batch_size, seq_length, hidden_size])
    print(f"输入形状: {test_input.shape}")

    # 使用自定义层归一化
    custom_ln = MyDense(epsilon=1e-3)
    custom_output = custom_ln(test_input)
    print(f"自定义层输出形状: {custom_output.shape}")

    # 使用标准层归一化
    standard_ln = tf.keras.layers.LayerNormalization(epsilon=1e-3)
    standard_output = standard_ln(test_input)
    print(f"标准层输出形状: {standard_output.shape}")

    # 计算差异
    absolute_diff = tf.abs(custom_output - standard_output)
    max_diff = tf.reduce_max(absolute_diff)
    mean_diff = tf.reduce_mean(absolute_diff)

    print(f"最大绝对差异: {max_diff.numpy():.6e}")
    print(f"平均绝对差异: {mean_diff.numpy():.6e}")

    # 检查是否几乎相同
    tolerance = 1e-6
    if max_diff < tolerance:
        print("✓ 测试通过！自定义层与标准层输出几乎相同")
    else:
        print("✗ 测试失败！输出存在明显差异")

    return custom_output, standard_output, max_diff, mean_diff



# 运行测试
if __name__ == "__main__":
    # 基本功能测试
    custom_out, standard_out, max_diff, mean_diff = test_layer_normalization()


测试层归一化实现...
输入形状: (4, 6, 8)
自定义层输出形状: (4, 6, 8)
标准层输出形状: (4, 6, 8)
最大绝对差异: 1.929823e+00
平均绝对差异: 3.381391e-01
✗ 测试失败！输出存在明显差异


2. 使用自定义训练循环训练模型来处理Fashion MNIST数据集（13_神经网络介绍 里用的数据集）：

    - a.显示每个轮次、迭代、平均训练损失和每个轮次的平均精度（在每次迭代中更新），以及每个轮次结束时的验证损失和精度。
    - b.尝试对上面的层和下面的层使用具有不同学习率的不同优化器。

In [59]:
# 自定义循环训练模型,自定义块和自定义模型
# 先自定义循环块
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self,n_layers,n_neurons=2, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = [tf.keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal") for _ in range(n_layers)]

    def call(self, inputs, *args, **kwargs):
        z = inputs
        for layer in self.hidden1:
            z = layer(z)

        return inputs + z

# 自定义模型
class ResidualRegressor(tf.keras.Model):
    def __init__(self,output_dim, **kwargs):
        super().__init__(**kwargs)
        # 第一个全连接层，将2维输入扩展到30维
        self.input_layer = tf.keras.layers.Dense(30, activation="relu")
        self.block = ResidualBlock(2)
        self.out = tf.keras.layers.Dense(output_dim, activation="softmax")
    def call(self, inputs, *args, **kwargs):
        # 首先将2维输入扩展到30维
        z = self.input_layer(inputs)
        z = self.block(inputs)
        return self.out(z)

In [57]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = iris.target
# 数据标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train,X_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=42)

In [60]:
model = ResidualRegressor(3)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # 适用于整数标签
    metrics=['accuracy']
)
model.fit(X_train, y_train, epochs=3, validation_split=0.1, verbose=1)

Epoch 1/3
4/4 [==============================] - 2s 86ms/step - loss: 1.1620 - accuracy: 0.5741 - val_loss: 1.1991 - val_accuracy: 0.5000
Epoch 2/3
4/4 [==============================] - 0s 19ms/step - loss: 1.1542 - accuracy: 0.6019 - val_loss: 1.1922 - val_accuracy: 0.5000
Epoch 3/3
4/4 [==============================] - 0s 18ms/step - loss: 1.1463 - accuracy: 0.6019 - val_loss: 1.1853 - val_accuracy: 0.5833


In [61]:
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 34ms/step - loss: 1.1884 - accuracy: 0.5667


[1.1883660554885864, 0.5666666626930237]